# HW Multi Location Vacuum Problem
We want to add mobility of the vacuum robot to clean the place. By expanding to two dimensional space and up to 9-places, the problem may be more complex.
<img src="images/hw2-1_multi_location_vacuum_problem.png">

In this question, you need to define the problem by yourself. The problem is limited by following rule:
- The vacuum robot agent can go **(Left, Right, Up, Down)** and clean the place by **Sucking** action.
- The place can be index 0~8 from left-top to right-bottom.
- The goal is that: there is no dirt in all place.
- If robot is located on edge of places, the going outside action **is not allowed**.
- If robot is located on the clean place, the clean action **is not allowed**.


## 1. Import python package

In [2]:
import math
import sys

# For some data structure implementation
import heapq
from collections import defaultdict, deque, Counter

## 2. Problem class definition

In [7]:
class Problem(object):
    def __init__(self, initial=None, goal=None, **other_keywords):
        """Specify the initial and goal states.
        Subclasses can use other keywords if they want."""
        self.__dict__.update(initial=initial, goal=goal, **other_keywords) 

    def actions(self, state):           raise NotImplementedError
    def result(self, state, action):    raise NotImplementedError
    def is_goal(self, state):           return state == self.goal
    def step_cost(self, s, action, s1): return 1
    def h(self, node):                  return 0

## 3. Node definition

In [8]:
class Node:
    '''A Node in a search tree.'''
    def __init__(self, state, parent=None, action=None, path_cost=0):
        # __dict__ store this object's all attributes
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)
    
    '''All Reserve words are not introduced here. If you are interest in them, please Google them'''
    # __repr__ is a built-in function used to compute the '''official''' string reputation of an object.
    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.state < other.state
    
failure = Node('failure', path_cost=float('inf')) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=float('inf')) # Indicates iterative deeepening search was cut off.

def expand(problem, node):
    '''Expand a node, generating the children nodes.'''
    s = node.state
    for action in problem.actions(s): 
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.step_cost(s, action, s1)
        yield Node(s1, node, action, cost)
        

def path_actions(node):
    '''The sequence of actions to get to this node.'''
    if node.parent is None:
        return []
    else: 
        return path_actions(node.parent) + [node.action]


def path_states(node):
    '''The sequence of states to get to this node.'''
    if node.parent is None:
        return ([] + [node.state])
    else:
        return (path_states(node.parent)) + [node.state]


def path(node):
    '''Alternating states and actions to get to this node.'''
    if node.parent is None:
        return ([] + [node.state])
    else:
        return (path(node.parent) + [node.action] ) + [node.state]

## 4. Search Algorithms

In [9]:
FIFOQueue = deque 
LIFOQueue = list  #stack

def depth_limited_search(problem, limit=20):
    "Search deepest nodes in the search tree first."
    frontier = LIFOQueue([Node(problem.initial)])
    solution = failure
    while frontier:
        node = frontier.pop()
        if len(node) > limit:
            solution = cutoff
        else:
            for child in expand(problem, node):
                if problem.is_goal(child.state):
                    return child
                frontier.append(child)
    return solution

def breadth_first_search(problem):
    "Search shallowest nodes in the search tree first."
    frontier = FIFOQueue([Node(problem.initial)])
    reached = set()
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            s = child.state
            if s not in reached:
                reached.add(s)
                frontier.appendleft(child)
    return failure
        

In [10]:
class CountCalls:
    """Delegate all attribute accesses to the object, and count them in ._counts"""
    def __init__(self, obj):
        self._object = obj
        self._counts = Counter()
        
    def __getattr__(self, attr):
        self._counts[attr] += 1
        return getattr(self._object, attr)
        
def report(searchers, problems):
    "Show metrics for each searcher on each problem."
    for searcher in searchers:
        print(searcher.__name__ + ':')
        total_counts = Counter()
        for p in problems:
            prob   = CountCalls(p)
            soln   = searcher(prob)
            counts = prob._counts; 
            counts.update(len=len(path_actions(soln)), cost=soln.path_cost)
            total_counts += counts
            report_line(counts, type(p).__name__)
        report_line(total_counts, 'TOTAL\n')
        
def report_line(counts, name):
    "Print one line of the report."
    print('{:9,d} explored |{:7,d} goal |{:5.0f} cost |{:3d} steps | {}'
          .format(counts['result'], counts['is_goal'], 
                  counts['cost'], counts['len'], name))

# MultiLocationVacuumProblem
<font color="red">
    <h3>Fill in the ??? and verify the code to implement BFS and DFS for this problem. </h3>
</font>

In [47]:
dirt  = '*'
clean = ' '

# robot location : at int(state[0])

def board8(state, fmt=('{} {} {}\n{} {} {}\n{} {} {}\n')):
    "A string representing an 8-puzzle board"
    loc = int(state[0])
    print("loc "+ str(loc)+"\n")
    dirt = list(state[1:]) # from list[1] to list[n]
    if dirt[loc] == clean: dirt[loc] = '◯' #dirt : for object board8 to record.  robot && clean
    else: dirt[loc] = '⊕'               # robot && dirt
    return fmt.format(*dirt)

class MultiLocationVacuumProblem(Problem):    
    def actions(self, state): 
        loc = int(state[0]) # robot location
        dirt = state[1:]    # record map dirt information 
        
        action_set = ['L', 'R', 'U', 'D', 'S']
        if loc in (0, 3, 6): action_set.remove('L') # 在左邊：不能再left了
        if loc in (0, 1, 2): action_set.remove('U')
        if loc in (6, 7, 8): action_set.remove('D')
        if loc in (2, 5, 8): action_set.remove('R')
        if dirt[loc] == clean: action_set.remove('S')
        
        return (tuple(action_set)) # we have these actions to choose!

    
    def is_goal(self, state):
        '''
        According to the goal described in top of this file, 
        the function will return true or false that whether
        the dirt is existed in the any state.
        
        return ???
        '''
        # true if dirt[] = clean
        # for dirt_item in dirt:
        for i in range (9):
          # print(dirt_item)
          if state[i+1]==dirt: #dirt_item == dirt:
            # print("false, dirt in "+str(i)+"\n")
            return False
        print("Success!\n")
        return True
    
    def result(self, state, action):   
        loc = int(state[0])
        dirt = list(state[1:])
        
        if action == 'L':
            # do something when action == Left move
            loc = loc -1
            return tuple(str(loc))+ tuple(dirt)
        elif action == 'R':
            # do something when action == Right move
            loc = loc +1
            return tuple(str(loc))+ tuple(dirt)
        elif action == 'U':
            # do something when action == Up move
            loc = loc -3
            return tuple(str(loc))+ tuple(dirt)
        elif action == 'D':
            # do something when action == Down move
            loc = loc +3
            return tuple(str(loc))+ tuple(dirt)
        elif action == 'S':
            # do something when action == Suck
            dirt[loc] = clean
            return tuple(str(loc))+ tuple(dirt)
        else: raise ValueRrror('unknown action: ' + action)



In [48]:
# Test the problem declaration is correct or not
initial_state = (0, '*', ' ', '*', ' ', '*', ' ', '*', ' ', '*')


p1 = MultiLocationVacuumProblem(initial_state)
print(p1.initial)
p1.result(p1.initial, 'D')
# p1.result(p1.initial, 'S')
# p1.result(p1.initial, 'R')
# p1.result(p1.initial, 'R')

(0, '*', ' ', '*', ' ', '*', ' ', '*', ' ', '*')


('3', '*', ' ', '*', ' ', '*', ' ', '*', ' ', '*')

In [50]:
initial_state = (3, ' ', '*', '*', ' ', '*', ' ', '*', ' ', '*')
p1 = MultiLocationVacuumProblem(initial=initial_state)

# result_graph = depth_limited_search(problem=p1, limit=20)
result_graph = breadth_first_search(problem=p1)
# Take a look the state sequence of the result
for s in path_states(result_graph):
    # print(s)
    print(board8(s))

Success!

loc 3

  * *
◯ *  
*   *

loc 4

  * *
  ⊕  
*   *

loc 4

  * *
  ◯  
*   *

loc 1

  ⊕ *
     
*   *

loc 1

  ◯ *
     
*   *

loc 2

    ⊕
     
*   *

loc 2

    ◯
     
*   *

loc 5

     
    ◯
*   *

loc 8

     
     
*   ⊕

loc 8

     
     
*   ◯

loc 7

     
     
* ◯  

loc 6

     
     
⊕    

loc 6

     
     
◯    



In [51]:
report([breadth_first_search, depth_limited_search], [p1])

breadth_first_search:
Success!

      822 explored |    279 goal |   12 cost | 12 steps | MultiLocationVacuumProblem
      822 explored |    279 goal |   12 cost | 12 steps | TOTAL

depth_limited_search:
Success!

   68,135 explored | 68,135 goal |   21 cost | 21 steps | MultiLocationVacuumProblem
   68,135 explored | 68,135 goal |   21 cost | 21 steps | TOTAL

